# Data Filtering Presets and User Submitted Substance Type Filtering

In [28]:
import pandas as pd 

dataset = pd.read_csv('2024.csv')
print(f"Total contaminations for dataset: {len(dataset)}")

FileNotFoundError: [Errno 2] No such file or directory: '2024.csv'

**⭐ Normalize the 'Substance Unit' column to lowercase and strip whitespace, count 'gallons' verses all other measurement units, and print the results**

In [16]:
unit_col = dataset['Substance Unit'].astype(str).str.lower().str.strip()

gallon_count = (unit_col == 'gallons').sum()
other_count = (unit_col != 'gallons').sum()

print(f"Number of spills using gallons as substance unit: {gallon_count}")
print(f"Number of spills using other units (ie pounds) as substance unit: {other_count} \n")

Number of spills using gallons as substance unit: 2939
Number of spills using other units (ie pounds) as substance unit: 81 



**⭐ See how many items in the dataset are missing critical location inputs**

In [17]:
missing_count = dataset['Latitude'].isna() | dataset['Longitude'].isna()
print(f"Rows with missing Latitude or Longitude in dataset: {missing_count.sum()} \n")

Rows with missing Latitude or Longitude in dataset: 1056 



**⭐ Calculate the sum of all gallons released in the dataset as well as the average number of gallons released per contamination. Only gallons are considered, not pounds or null measurements**

In [19]:
#filter rows where 'substance unit' is gallons
gallons_dataset = dataset[dataset['Substance Unit'].str.lower().str.strip() == 'gallons']
total_gallons = gallons_dataset['Quantity Released'].sum()
average_gallons = gallons_dataset['Quantity Released'].mean()

print(f"Total gallons released in data: {total_gallons}")
print(f"Average gallons released per event in dataset: {average_gallons} \n")

Total gallons released in data: 518784.57899999997
Average gallons released per event in dataset: 176.51737972099352 



**⭐ Provide information about what types of substances are included in the dataset** 

In [20]:
print(f"Included substance types in dataset: {dataset['Substance Type'].unique().tolist()}")

Included substance types in dataset: ['Noncrude Oil', 'Hazardous Substance', 'Unknown', 'Process Water', 'Crude Oil', 'Extremely Hazardous Substance', nan]


**⭐ Functionality to allow a user to change the dataset to only account for a given substance**

substance_counts = dataset['Substance Type'].value_counts()
valid_substances = list(substance_counts.index)
substances = dataset['Substance Type'].dropna().unique()
clean_list = ",\n".join(substances)

filtered_substance_dataset=[]

print(f"Would you like to filter on a specific substance type? Options are: \n\n{clean_list}\n")

while True:
    print("Click enter to not use any filter!")
    user_input = input('Substance:')
    
    if user_input == "":
        print("Got it! I'll search using all substance types.")
        break
        
    elif user_input in valid_substances:
        filtered_count = (dataset['Substance Type'] == user_input).sum()
        print(f"Awesome! I'll filter the dataset using '{user_input}'. We'll be searching across {filtered_count} contaminations.")
        break
        
    else:
        print("Sorry, that input isn’t in the list. Please choose one of the options above exactly as written.")

if user_input:
    filtered_substance_dataset = dataset[dataset['Substance Type'] == user_input]  
    dataset=filtered_substance_dataset

**⭐ Filter out dataset to only include contaminations that occured in the ocean**

In [ ]:
from global_land_mask import globe

def get_ocean_rows(dataframe):
    ocean_rows = []
    for _, row in dataframe.iterrows():
        if pd.isna(row['Latitude']) or pd.isna(row['Longitude']):
            continue
        if not (-90 <= row['Latitude'] <= 90 and -180 <= row['Longitude'] <= 180):
            continue
        status = globe.is_land(row['Latitude'], row['Longitude'])
        if status == False:
            ocean_rows.append(row)
    ocean_df = pd.DataFrame(ocean_rows)
    print(f"There are {len(ocean_df)} reported ocean contaminations in your dataset")
    return ocean_df

In [ ]:
ocean_df = get_ocean_rows(dataset)